In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('./query_result.csv')
df

,match_id,m
0,36168,6.019
1,29014,0.094
2,37917,3.832
3,31304,6.717
4,32514,2.106
...,...,...
11950,33258,3.047
11951,37864,4.138
11952,29952,2.243
11953,29817,5.431


In [ ]:
df[]

In [10]:
pd.DataFrame(df['m'].quantile([0,0.1,0.2,0.25,0.3,0.4,0.5,0.6,0.7,0.75,0.8,0.9,0.95,0.99,0.995,0.996,0.997,0.998,0.999,1.0])).T

,0.000,0.100,0.200,0.250,0.300,0.400,0.500,0.600,0.700,0.750,0.800,0.900,0.950,0.990,0.995,0.996,0.997,0.998,0.999,1.000
m,0.004,1.521,2.0178,2.3085,2.5822,3.166,3.783,4.56,5.5198,6.0415,6.7602,8.9152,11.1924,17.12598,20.64743,22.3928,24.183362,28.920144,101.037784,583.234


In [1]:
"""
Connect senior:Mainland
"""
import pandas as pd
import json
import requests
def ssql(sql):
    admin_token = '2cebeb5c7edac5ef0d7c5a640e69fc7d43bf2cd6a24ce43dfee3dd33662c0bab'
    url = 'http://sensor.wb-intra.com/api/sql/query?token=%s&project=production' % admin_token
    data = {'q': sql, 'format': 'json'}
    req = requests.post(url,data)
    req_dec = req.content.decode()
    try:
        req_json = json.loads('[' + req_dec.replace('\n', ',')[:-1] + ']')
        df_d_id = pd.DataFrame(req_json)
        return df_d_id
    except:
        print(req_dec)

In [2]:
df = ssql("""--安卓新用户的游戏转化率
SELECT t1.date,
       count(t1.distinct_id) AS `新增用户数`,
       count(t2.distinct_id) AS `登录用户数`,
       count(t2.distinct_id) /count(t1.distinct_id) AS `注册登录转化率`,
       count(t3.distinct_id) AS `用户数`,
       count(t3.distinct_id)/count(t1.distinct_id) AS `实名认证触发率`,
       count(t4.distinct_id) AS `实名认证成功用户数`,
       count(t4.distinct_id)/count(t3.distinct_id) AS `实名认证成功转化率`,
       count(t5.distinct_id) AS `点击进房用户数`,
       count(t5.distinct_id)/count(t3.distinct_id) AS `新用户(触发实名认证)进房率`,
       count(t6.distinct_id) AS `分配房间用户数`,
       count(t6.distinct_id)/count(t3.distinct_id) AS `新用户(触发实名认证)匹配率`,
       count(t7.distinct_id) AS`开始游戏用户数`,
       count(t7.distinct_id)/count(t3.distinct_id) AS `新用户(触发实名认证)游戏率`,
       count(t8.distinct_id) AS`结束游戏用户数`,
       count(t8.distinct_id)/count(t3.distinct_id) AS `新用户(触发实名认证)完局率`
FROM
  (SELECT date,distinct_id
   FROM events
   WHERE event = 'register'--注册
     AND appId = '20014'
     AND date BETWEEN '2020-10-20' AND current_date()
   GROUP BY 1,
            2)t1
LEFT JOIN
  (SELECT date,distinct_id
   FROM events
   WHERE event = 'login'--登录
     AND appId = '20014'
     AND date BETWEEN '2020-10-20' AND current_date()
   GROUP BY 1,
            2)t2 ON t1.date = t2.date
AND t1.distinct_id = t2.distinct_id
LEFT JOIN
  (SELECT date,distinct_id
   FROM events
   WHERE event = 'action'--防沉迷实名弹窗
     AND appId = '20014'
      AND date BETWEEN '2020-10-20' AND current_date()
     AND op_type = 'anti_indulgence_popup'
     AND pop_type = 'verified_mode_popup'
   GROUP BY 1,
            2)t3 ON t1.date = t3.date
AND t1.distinct_id = t3.distinct_id
AND t2.distinct_id = t3.distinct_id
LEFT JOIN
  (SELECT date,distinct_id
   FROM events
   WHERE event = 'regRealName'--实名认证成功
     AND appId = '20014'
     AND date BETWEEN '2020-10-20' AND current_date()
     AND RESULT= 'success'
   GROUP BY 1,
            2)t4 ON t1.date = t4.date
AND t1.distinct_id = t4.distinct_id
AND t2.distinct_id = t4.distinct_id
AND t3.distinct_id = t4.distinct_id
LEFT JOIN
  (SELECT date,distinct_id
   FROM events
   WHERE event = 'cocosEvent'--进房
     AND gameTypeId = 1800
     AND cocos_event = 'enter_room'
      AND date BETWEEN '2020-10-20' AND current_date()
   GROUP BY 1,
            2)t5 ON t1.date = t5.date
AND t1.distinct_id= t5.distinct_id
AND t2.distinct_id = t5.distinct_id
AND t3.distinct_id = t5.distinct_id
AND t4.distinct_id = t5.distinct_id
LEFT JOIN
  (SELECT date,distinct_id
   FROM events
   WHERE event = 'roomAllocate'--进房
     AND gameTypeId = 1800
      AND date BETWEEN '2020-10-20' AND current_date()
   GROUP BY 1,
            2)t6 ON t1.date = t6.date
AND t1.distinct_id= t6.distinct_id
AND t2.distinct_id= t6.distinct_id
AND t3.distinct_id = t6.distinct_id
AND t4.distinct_id = t6.distinct_id
LEFT JOIN
  (SELECT date,distinct_id
   FROM events
   WHERE event = 'gameStart'--游戏开始
     AND gameTypeId = 1800
     AND game_played = 0
      AND date BETWEEN '2020-10-20' AND current_date()
   GROUP BY 1,
            2)t7 ON t1.date = t7.date
AND t1.distinct_id= t7.distinct_id
AND t2.distinct_id= t7.distinct_id
AND t3.distinct_id = t7.distinct_id
AND t4.distinct_id = t7.distinct_id
AND t6.distinct_id = t7.distinct_id
LEFT JOIN
  (SELECT date,distinct_id
   FROM events
   WHERE event = 'gameOver'--游戏结束
     AND gameTypeId = 1800
      AND date BETWEEN '2020-10-20' AND current_date()
   GROUP BY 1,
            2)t8 ON t1.date = t8.date
AND t1.distinct_id= t8.distinct_id
AND t2.distinct_id= t8.distinct_id
AND t3.distinct_id = t8.distinct_id
AND t4.distinct_id = t8.distinct_id
AND t6.distinct_id = t8.distinct_id
AND t7.distinct_id = t8.distinct_id
GROUP BY 1
ORDER BY 1""")
df

{"error":"java.lang.NumberFormatException"}
